# Building Loyalty Dataset for 2019-04  
### Date 2019-05-02  

First step is import data from Parse.ly's AWS bucket for the relevant time period into Google BigQuery.  
The key and passphrase for these are with rachit.kinger@jpimedia.co.uk. More key and passwords can be obtained by reaching out directly to Parse.ly.  
This data is best imported via GCP's GUI. Go to `GCP > Storage > Transfer` and the steps from there should be obvious.  
Remember to specify file filters to download data only for the relevant months.  

Once this data has been download the follow these steps to import into a single 'large' database. DO NOT use this database for analysis. Only around 30% of this dataset is useful for us. This database will have a column called `action` which has mostly two values, `pageview` or `heartbeat`. The `pageview` is what is of interest to us and it occupies only 30% of the database.  



In [ ]:
# the following commands were run from bash using gsutil
# create empty table with the rawdata schema
%%bash
bq mk -t --schema ./parsely_rawdata_schema.json \
--time_partitioning_type=DAY --time_partitioning_field ts_action \
--require_partition_filter=TRUE --clustering_fields='apikey,action' \
parsely.rawdata

In [ ]:
# the following commands were run from bash using gsutil
# use the following command to load the raw parsely data into the the empty table created in previous step
%%bash
bq load --source_format=NEWLINE_DELIMITED_JSON --max_bad_records=1000 --ignore_unknown_values parsely.rawdata gs://parsely/*

In [1]:
# use oauth to authenticate access into GCP and bigquery
# toggle this variable depending on whether you are on a machine that has an Internet browser or not
launch_browser = True 
# The `launch_browser` boolean variable indicates if a local server is used
# as the callback URL in the auth flow. A value of `True` is recommended,
# but a local server does not work if accessing the application remotely,
# such as over SSH or from a remote Jupyter notebook.

from google_auth_oauthlib import flow

# if you are not Rachit Kinger please build your own oauth access client id and key and set the 
# path in the function below to your own client secret

appflow = flow.InstalledAppFlow.from_client_secrets_file(
    '/home/rachit/gdrive/GCP/oauth_client_key/data-team-rachit-kinger-desktop-apps.json',
    scopes = ['https://www.googleapis.com/auth/bigquery'])

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()
    
credentials = appflow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=306244342965-pv1n0vhib5nv2ks66pnj59csdf8cqor0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=B66ovDQNZ96VIWP5cRblogvPYIyRfD&access_type=offline


In [2]:
# access bigquery
from google.cloud import bigquery
project_id = "bigquery-test-165213"
client = bigquery.Client(project = project_id, credentials = credentials)

In [4]:
#create new dataset without the non-pageviews data
job_config = bigquery.QueryJobConfig()
#set destination table
table_ref = client.dataset('parsely').table('raw_data_for_further_analysis')
job_config.destination = table_ref

sql = '''
SELECT
  apikey,
  event_id,
  flags_is_amp,
  ip_city,
  ip_lat,
  ip_lon,
  metadata_full_content_word_count, --newly added on 2019-04-02
  metadata_page_type,
  TIMESTAMP_MILLIS(metadata_pub_date_tmsp) AS metadata_pub_date_tmsp, --newly added on 2019-04-02
  TIMESTAMP_MILLIS(metadata_save_date_tmsp) AS metadata_save_date_tmsp, --newly added on 2019-04-02
  metadata_post_id, --newly added on 2019-04-02
  metadata_section,
  metadata_tags,
  metadata_title,
  ref_category,
  ref_domain,
  session,
  session_id, 
  session_initial_referrer,
  ts_action,
  ua_device,
  ua_devicetype, 
  url_clean,
  visitor,
  visitor_site_id,
  visitor_network_id
FROM
  `bigquery-test-165213.parsely.rawdata`
WHERE
  ts_action < TIMESTAMP("2019-05-01") --FOR APR-2019
  AND action = "pageview"
  AND visitor_site_id NOT IN ("", "OPTOUT")
'''

#run the query
query_job = client.query(
sql,
# location must match that of the dataset(s) referenced in the query
# and that of the destination table
location = 'EU',
job_config=job_config)

query_job.result() # waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))


Query results loaded to table /projects/bigquery-test-165213/datasets/parsely/tables/raw_data_for_further_analysis


The next set of steps tries to impute session_ids in order to determine how frequently users visit our sites. The main reason why sessions ids are not captured is because AMP pages do not return session data however, Parse.ly does manage to assign `visitor_site_id` to the visitors of AMP pages. To calculate the number of sessions a user has had we will assume that a session lasts maximum for 30 minutes and any pageviews outside of these time gaps are part of different sessions.  

In [3]:
# create a separate dataset which has imputed session_ids where session_ids do not exist

job_config = bigquery.QueryJobConfig()
# assign destination table
table_ref = client.dataset('parsely').table('imputed_session_ids_for_null_sessions')
job_config.destination = table_ref

sql = '''
  /*
REMEMBER TO CHANGE NUMERICAL VALUE IN THE LAST LINE OF QUERY:
EXTRACT(MONTH FROM ts_action) = 4 <--- THIS NUMBER SHOULD MONTH OF ANALYSIS
loyalty analysis
creating table 1
where session_id = NULL
where visitor_site_id != BLANK
This table will create another table which has the same schema as
raw_data_for_further_analysis but will compute session_id for those users who have NULL as session_ids
this table will then be unioned with table 2 where session_id != NULL and visitor_site_id != BLANK

NOTE: All rows where visitor_site_id IN ("", "OPTOUT") will be removed for analysis
*/

SELECT
  * EXCEPT (ts_action,
    session_change,
    time_diff,
    previous_ts),
  SUM(session_change) OVER (PARTITION BY visitor_site_id ORDER BY ts_action ASC) AS session_id
FROM (
  SELECT
    *,
    CASE
      WHEN time_diff IS NULL THEN 1
      WHEN time_diff > 1800 THEN 1
      ELSE 0
    END AS session_change
  FROM (
    SELECT
      *,
      TIMESTAMP_DIFF(ts_action, previous_ts, SECOND) AS time_diff
    FROM (
      SELECT
        visitor_site_id,
        event_id,
        ts_action,
        LAG(ts_action) OVER (PARTITION BY visitor_site_id ORDER BY ts_action ASC) AS previous_ts
      FROM
        `bigquery-test-165213.parsely.raw_data_for_further_analysis`
      WHERE
        session_id IS NULL
        AND EXTRACT(MONTH FROM ts_action) = 4) AS w_prev_ts) AS w_time_diff) AS w_sess_change
'''

# run the query
query_job = client.query(
sql,
# location must match that of the dataset(s) referenced in the query
# and that of the destination table
location = 'EU',
job_config=job_config)

query_job.result() # waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/bigquery-test-165213/datasets/parsely/tables/imputed_session_ids_for_null_sessions


In [4]:
# reinserting imputed session ids back into raw_data_for_march
job_config = bigquery.QueryJobConfig()
table_ref = client.dataset('parsely').table('april_with_session_ids')
job_config.destination = table_ref

sql = '''
  /*
REMEMBER TO CHANGE NUMERICAL VALUE IN THE QUERY:
EXTRACT(MONTH FROM ts_action) = 4 <--- THIS NUMBER SHOULD MONTH OF ANALYSIS
Inserting imputed session_ids for null sessions back into
raw data
Using LEFT JOIN on rawdata with imputed rawdata*/

SELECT
  main.* EXCEPT(session_id),
  CASE
    WHEN main.session_id IS NULL THEN imputed.session_id
    ELSE main.session_id
  END AS session_id
FROM (
  SELECT
    *
  FROM
    `bigquery-test-165213.parsely.raw_data_for_further_analysis`
  WHERE EXTRACT(MONTH FROM ts_action) = 4 ) AS main
LEFT JOIN (
  SELECT
    event_id,
    session_id
  FROM
    `bigquery-test-165213.parsely.imputed_session_ids_for_null_sessions`) AS imputed
ON
  main.event_id = imputed.event_id
'''

# run the query
query_job = client.query(
sql,
# location must match that of the dataset(s) referenced in the query
# and that of the destination table
location = 'EU',
job_config=job_config)

query_job.result() # waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/bigquery-test-165213/datasets/parsely/tables/april_with_session_ids


In [5]:
# determine monthly loyalty segment for each visitor
# add a new column to an existing table and update values based on ts_action


job_config = bigquery.QueryJobConfig()
table_ref = client.dataset('parsely').table('loyalty_april')
job_config.destination = table_ref

sql = '''
/* REMEMBER TO CHANGE THE FROM TABLE NAME AND THE TABLE_REF NAME FROM ABOVE */
SELECT
  * EXCEPT(flags_is_amp,
    session,
    visitor),
  COUNT(DISTINCT session_id) OVER(PARTITION BY visitor_site_id) AS number_of_sessions,
  CASE
    WHEN (COUNT(DISTINCT session_id) OVER(PARTITION BY visitor_site_id)) = 1 THEN "Flyby (1 visit)"
    WHEN (COUNT(DISTINCT session_id) OVER(PARTITION BY visitor_site_id)) BETWEEN 2
  AND 5 THEN "Casual (2-5 visits)"
    WHEN (COUNT(DISTINCT session_id) OVER(PARTITION BY visitor_site_id)) BETWEEN 6 AND 15 THEN "Loyal (6-15 visits)"
    WHEN (COUNT(DISTINCT session_id) OVER(PARTITION BY visitor_site_id)) BETWEEN 16
  AND 24 THEN "Brand Lover (16-24 visits)"
    WHEN (COUNT(DISTINCT session_id) OVER(PARTITION BY visitor_site_id)) >= 25 THEN "Habitual (25+ visits)"
    ELSE "Uncategorised"
  END AS Loyalty_Segment,
  CASE
    WHEN apikey IN ('arbroathherald.co.uk',  'ballymenatimes.com',  'ballymoneytimes.co.uk',  'banbridgeleader.co.uk',  'banburyguardian.co.uk',  'batleynews.co.uk',  'bedfordtoday.co.uk',  'belpernews.co.uk',  'berkhamstedtoday.co.uk',  'berwick-advertiser.co.uk',  'berwickshirenews.co.uk',  'bexhillobserver.net',  'biggleswadetoday.co.uk',  'blackpoolgazette.co.uk',  'bognor.co.uk',  'bostonstandard.co.uk',  'brechinadvertiser.co.uk',  'bridlingtonfreepress.co.uk',  'brighouseecho.co.uk',  'brightonandhoveindependent.co.uk',  'buchanobserver.co.uk',  'buckinghamtoday.co.uk',  'bucksherald.co.uk',  'burnleyexpress.net',  'buteman.co.uk',  'buxtonadvertiser.co.uk',  'carlukegazette.co.uk',  'carrickfergustimes.co.uk',  'carricktoday.co.uk',  'chad.co.uk',  'chichester.co.uk',  'chorley-guardian.co.uk',  'clitheroeadvertiser.co.uk',  'colerainetimes.co.uk',  'crawleyobserver.co.uk',  'cumbernauld-news.co.uk',  'daventryexpress.co.uk',  'deesidepiper.co.uk',  'derbyshiretimes.co.uk',  'derryjournal.com',  'dewsburyreporter.co.uk',  'dirtbikerider.com',  'doncasterfreepress.co.uk',  'donsidepiper.co.uk',  'dromoreleader.co.uk',  'dunstabletoday.co.uk',  'eastbourneherald.co.uk',  'eastwoodadvertiser.co.uk',  'edinburghnews.scotsman.com',  'ellontimes.co.uk',  'falkirkherald.co.uk',  'farminglife.com',  'fifetoday.co.uk',  'fileymercury.co.uk',  'fleetwoodtoday.co.uk',  'forfardispatch.co.uk',  'fraserburghherald.co.uk',  'gainsboroughstandard.co.uk',  'gallowaygazette.co.uk',  'garstangcourier.co.uk',  'glasgowsouthandeastwoodextra.co.uk',  'guideandgazette.co.uk',  'halifaxcourier.co.uk',  'harboroughmail.co.uk',  'harrogateadvertiser.co.uk',  'hartlepoolmail.co.uk',  'hastingsobserver.co.uk',  'hawick-news.co.uk',  'hebdenbridgetimes.co.uk',  'hemeltoday.co.uk',  'hemsworthandsouthelmsallexpress.co.uk',  'horncastlenews.co.uk',  'hucknalldispatch.co.uk',  'ilkestonadvertiser.co.uk',  'inverurieherald.co.uk',  'kenilworthweeklynews.co.uk',  'kincardineshireobserver.co.uk',  'kirkintilloch-herald.co.uk',  'kirriemuirherald.co.uk',  'lancasterguardian.co.uk',  'larnetimes.co.uk',  'leamingtoncourier.co.uk',  'leighobserver.co.uk',  'leightonbuzzardonline.co.uk',  'lep.co.uk',  'leylandguardian.co.uk',  'linlithgowgazette.co.uk',  'lisburntoday.co.uk',  'littlehamptongazette.co.uk',  'londonderrysentinel.co.uk',  'longridgenews.co.uk',  'louthleader.co.uk',  'lurganmail.co.uk',  'lutontoday.co.uk',  'lythamstannesexpress.co.uk',  'marketrasenmail.co.uk',  'matlockmercury.co.uk',  'mearnsleader.co.uk',  'meltontimes.co.uk',  'midhurstandpetworth.co.uk',  'midlothianadvertiser.co.uk',  'midsussextimes.co.uk',  'midulstermail.co.uk',  'milngavieherald.co.uk',  'miltonkeynes.co.uk',  'mirfieldreporter.co.uk',  'montrosereview.co.uk',  'morleyobserver.co.uk',  'morpethherald.co.uk',  'motherwelltimes.co.uk',  'newsguardian.co.uk',  'newsletter.co.uk',  'newspostleader.co.uk',  'newtownabbeytoday.co.uk',  'northamptonchron.co.uk',  'northantstelegraph.co.uk',  'northumberlandgazette.co.uk',  'northyorkshirenews.com',  'pendletoday.co.uk',  'peterboroughtoday.co.uk',  'pocklingtonpost.co.uk',  'pontefractandcastlefordexpress.co.uk',  'portadowntimes.co.uk',  'portsmouth.co.uk',  'retfordtoday.co.uk',  'ripleyandheanornews.co.uk',  'ripongazette.co.uk',  'rugbyadvertiser.co.uk',  'ryeandbattleobserver.co.uk',  'scotsman.com',  'sheffieldtelegraph.co.uk',  'shieldsgazette.com',  'shorehamherald.co.uk',  'skegnessstandard.co.uk',  'sleafordstandard.co.uk',  'southyorkshiretimes.co.uk',  'spenboroughguardian.co.uk',  'sthelensreporter.co.uk',  'stornowaygazette.co.uk',  'sunderlandecho.com',  'sussexexpress.co.uk',  'thametoday.co.uk',  'thescarboroughnews.co.uk',  'thesouthernreporter.co.uk',  'thestar.co.uk',  'thevisitor.co.uk',  'thornegazette.co.uk',  'tmxnews.co.uk',  'todmordennews.co.uk',  'tringtoday.co.uk',  'tyronetimes.co.uk',  'wakefieldexpress.co.uk',  'warwickcourier.co.uk',  'westsussextoday.co.uk',  'wetherbynews.co.uk',  'whitbygazette.co.uk',  'wigantoday.net',  'worksopguardian.co.uk',  'worthingherald.co.uk',  'wscountytimes.co.uk',  'yorkshireeveningpost.co.uk',  'yorkshirepost.co.uk') THEN "Newsbrand"
    ELSE "Not newsbrand"
  END AS Title_Type,
  CASE
    WHEN apikey IN ('edinburghnews.scotsman.com',  'lep.co.uk',  'portsmouth.co.uk',  'sunderlandecho.com',  'thestar.co.uk',  'yorkshireeveningpost.co.uk' ) THEN "Big Cities"
    WHEN apikey IN ('banburyguardian.co.uk',
    'buckinghamtoday.co.uk',
    'bucksherald.co.uk',
    'chichester.co.uk',
    'derbyshiretimes.co.uk',
    'eastbourneherald.co.uk',
    'falkirkherald.co.uk',
    'harboroughmail.co.uk',
    'harrogateadvertiser.co.uk',
    'hastingsobserver.co.uk',
    'lancasterguardian.co.uk',
    'leamingtoncourier.co.uk',
    'meltontimes.co.uk',
    'miltonkeynes.co.uk',
    'northamptonchron.co.uk',
    'northantstelegraph.co.uk',
    'peterboroughtoday.co.uk',
    'sheffieldtelegraph.co.uk',
    'worthingherald.co.uk',
    'wscountytimes.co.uk') THEN "Growth Market Weeklies"
    WHEN apikey IN ('blackpoolgazette.co.uk',  'burnleyexpress.net',  'chad.co.uk',  'derryjournal.com',  'dewsburyreporter.co.uk',  'doncasterfreepress.co.uk',  'halifaxcourier.co.uk',  'hartlepoolmail.co.uk',  'pontefractandcastlefordexpress.co.uk',  'shieldsgazette.com',  'thescarboroughnews.co.uk',  'wakefieldexpress.co.uk',  'wigantoday.net') THEN "Heartland"
    WHEN apikey IN ('inews.co.uk') THEN "i"
    WHEN apikey IN ("newsletter.co.uk",  "scotsman.com",  "yorkshirepost.co.uk") THEN "Large National/Regional"
    WHEN apikey IN ('batleynews.co.uk',
    'bedfordtoday.co.uk',
    'berwick-advertiser.co.uk',
    'berwickshirenews.co.uk',
    'bexhillobserver.net',
    'biggleswadetoday.co.uk',
    'bognor.co.uk',
    'bostonstandard.co.uk',
    'bridlingtonfreepress.co.uk',
    'buxtonadvertiser.co.uk',
    'carlukegazette.co.uk',
    'chorley-guardian.co.uk',
    'clitheroeadvertiser.co.uk',
    'cumbernauld-news.co.uk',
    'daventryexpress.co.uk',
    'dunstabletoday.co.uk',
    'farminglife.com',
    'fifetoday.co.uk',
    'gainsboroughstandard.co.uk',
    'garstangcourier.co.uk',
    'hemeltoday.co.uk',
    'kirkintilloch-herald.co.uk',
    'larnetimes.co.uk',
    'leightonbuzzardonline.co.uk',
    'leylandguardian.co.uk',
    'lisburntoday.co.uk',
    'littlehamptongazette.co.uk',
    'louthleader.co.uk',
    'lurganmail.co.uk',
    'lutontoday.co.uk',
    'lythamstannesexpress.co.uk',
    'matlockmercury.co.uk',
    'midsussextimes.co.uk',
    'midulstermail.co.uk',
    'milngavieherald.co.uk',
    'motherwelltimes.co.uk',
    'newsguardian.co.uk',
    'newspostleader.co.uk',
    'newtownabbeytoday.co.uk',
    'northumberlandgazette.co.uk',
    'pendletoday.co.uk',
    'portadowntimes.co.uk',
    'rugbyadvertiser.co.uk',
    'ryeandbattleobserver.co.uk',
    'shorehamherald.co.uk',
    'skegnessstandard.co.uk',
    'sleafordstandard.co.uk',
    'southyorkshiretimes.co.uk',
    'sthelensreporter.co.uk',
    'stornowaygazette.co.uk',
    'sussexexpress.co.uk',
    'thametoday.co.uk',
    'thesouthernreporter.co.uk',
    'tmxnews.co.uk',
    'whitbygazette.co.uk',
    'worksopguardian.co.uk') THEN "Mid-Size"
    WHEN apikey IN ('adperfect.com',  'belfastvibe.com',  'Business Directories',  'digitalkitbag.com',  'HTML5 - iNews',  'HTML5 - Scotsman',  'iAnnounce',  'Jobstoday',  'johnstonpress.co.uk',  'jplocalbusiness.co.uk',  'thesmartlist.co.uk',  'wow247.co.uk') THEN "Other Sites"
    WHEN apikey IN ('arbroathherald.co.uk',
    'ballymenatimes.com',
    'ballymoneytimes.co.uk',
    'banbridgeleader.co.uk',
    'belpernews.co.uk',
    'berkhamstedtoday.co.uk',
    'brechinadvertiser.co.uk',
    'brighouseecho.co.uk',
    'brightonandhoveindependent.co.uk',
    'buchanobserver.co.uk',
    'buteman.co.uk',
    'carrickfergustimes.co.uk',
    'carricktoday.co.uk',
    'colerainetimes.co.uk',
    'crawleyobserver.co.uk',
    'deesidepiper.co.uk',
    'dirtbikerider.com',
    'donsidepiper.co.uk',
    'dromoreleader.co.uk',
    'eastwoodadvertiser.co.uk',
    'ellontimes.co.uk',
    'fileymercury.co.uk',
    'fleetwoodtoday.co.uk',
    'forfardispatch.co.uk',
    'fraserburghherald.co.uk',
    'gallowaygazette.co.uk',
    'glasgowsouthandeastwoodextra.co.uk',
    'guideandgazette.co.uk',
    'hawick-news.co.uk',
    'hebdenbridgetimes.co.uk',
    'hemsworthandsouthelmsallexpress.co.uk',
    'horncastlenews.co.uk',
    'hucknalldispatch.co.uk',
    'ilkestonadvertiser.co.uk',
    'inverurieherald.co.uk',
    'kenilworthweeklynews.co.uk',
    'kincardineshireobserver.co.uk',
    'kirriemuirherald.co.uk',
    'leighobserver.co.uk',
    'linlithgowgazette.co.uk',
    'londonderrysentinel.co.uk',
    'longridgenews.co.uk',
    'marketrasenmail.co.uk',
    'mearnsleader.co.uk',
    'midhurstandpetworth.co.uk',
    'midlothianadvertiser.co.uk',
    'mirfieldreporter.co.uk',
    'montrosereview.co.uk',
    'morleyobserver.co.uk',
    'morpethherald.co.uk',
    'northyorkshirenews.com',
    'pocklingtonpost.co.uk',
    'retfordtoday.co.uk',
    'ripleyandheanornews.co.uk',
    'ripongazette.co.uk',
    'spenboroughguardian.co.uk') THEN "Small"
    ELSE "Unknown"
  END AS Title_Segment
FROM
  `bigquery-test-165213.parsely.april_with_session_ids`
'''
# run the query
query_job = client.query(
sql,
# location must match that of the dataset(s) referenced in the query
# and that of the destination table
location = 'EU',
job_config=job_config)

query_job.result() # waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))


Query results loaded to table /projects/bigquery-test-165213/datasets/parsely/tables/loyalty_april


In [6]:
job_config = bigquery.QueryJobConfig()
table_ref = client.dataset('parsely').table('loyalty_analysis_datastudio_each_row_is_pageview_partitioned_w_page_type')
job_config.destination = table_ref
## the following data needs to be appended to an existing table and not create a new table hence below command
job_config.write_disposition = "WRITE_APPEND"

sql = '''
/*ENSURE YOU HAVE SELECTED THE WRITE TABLE TO APPEND FROM */
SELECT 
* 
FROM 
`bigquery-test-165213.parsely.loyalty_april`
'''


query_job = client.query(
sql,
location = 'EU',
job_config = job_config)

query_job.result() # waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))


Query results loaded to table /projects/bigquery-test-165213/datasets/parsely/tables/loyalty_analysis_datastudio_each_row_is_pageview_partitioned_w_page_type


# Update the connection of the bigquery data into data study so that loyalty anlaysis report updates automatically  

# Clean-up tasks
Delete following tables  
- raw data
- extracted relevant data  
- imputed sessions data
- loyal jan/feb data  

